In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torchsummary

In [ ]:
from torch import nn
import torch, torchvision, os
from torchsummary import summary
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def load_data(file):
    with open(file, 'rb') as f:
        X_test = np.load(f)
        X_valid = np.load(f)
        X_train = np.load(f)
        Y_test = np.load(f)
        Y_valid = np.load(f)
        Y_train = np.load(f)
    return X_test, X_valid, X_train, Y_test, Y_valid, Y_train

In [ ]:
X_test, X_valid, X_train, Y_test, Y_valid, Y_train = load_data('/kaggle/input/dpl-project/features_labels_15000.npy')

print(f'X_train:{X_train.shape}, y_train:{Y_train.shape}')
print(f'X_valid:{X_valid.shape}, Y_valid:{Y_valid.shape}')
print(f'X_test:{X_test.shape}, y_test:{Y_test.shape}')

In [ ]:
labels = ['positive', 'neutral', 'negative']
for ind, label in enumerate(labels):
#     print('Total sample of',label,'=',resulty[resulty[:,ind]==1].shape[0])
    total = np.concatenate((Y_train[:,ind], Y_valid[:,ind], Y_test[:,ind])).sum()
    print('Total sample of',label,'=',total)

In [ ]:
# X_train_ = ['positive','neutral','negative']
# Y_train_ = ['positive','neutral','negative']
# for i in range(3):    
#     selected_indices = np.where(Y_train[:, i] == 1)[0]
#     selected_rows = Y_train[Y_train[:, i] == 1]
#     Y_train_[i] = np.array(selected_rows)[:1783,:]
#     print(Y_train_[i].shape)
#     selected_data = [X_train[ind,:,:,:] for ind in selected_indices[:1783]]
#     X_train_[i] = np.array(selected_data)
#     print(X_train_[i].shape)
# Y_train = np.concatenate((Y_train_[0], Y_train_[1],Y_train_[2]), axis=0)
# X_train = np.concatenate((X_train_[0], X_train_[1],X_train_[2]), axis=0)

In [ ]:
# Model flow
class define_model(nn.Module):

    # Define layers
    def __init__(self, num_emotions):
        super().__init__()

        transformer_layer = nn.TransformerEncoderLayer(
            d_model=40, #####################
            nhead=4,
            dim_feedforward=512,
            dropout=0.4,
            activation='relu'
        )
        self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=4)

        #maxpool: reshape (width, height)
        #conv: reshape (channel)
        conv2d_layer = nn.Sequential(
            nn.Conv2d(
                in_channels=2,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(16),#######
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(32),#######
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(64),#######
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
        )
        self.conv2Dblock1 = nn.Sequential(
            nn.Conv2d(
                in_channels=2,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
        )
        self.conv2Dblock2 = conv2d_layer

        self.fc1_layer = nn.Linear(1792*2+40, 1800)
        self.act1 = nn.ReLU()
#         self.fc2_layer = nn.Linear(1800, 9)
#         self.act1 = nn.ReLU()
        self.fc2_layer = nn.Linear(1800, num_emotions)
        self.softmax_out = nn.Softmax(dim=1)

    def forward(self, x):
        conv2d_embedding1 = self.conv2Dblock1(x)
        conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim = 1)

        conv2d_embedding2 = self.conv2Dblock2(x)
        conv2d_embedding2 = torch.flatten(conv2d_embedding2, start_dim = 1)

        x_maxpool = self.transformer_maxpool(x)
        # x_maxpool_reduced = torch.squeeze(x_maxpool,1) 
        x_maxpool_reduced = x_maxpool.resize(x_maxpool.shape[0], 40, 450) 
        x = x_maxpool_reduced.permute(2,0,1) #rearrange dims
        transformer_output = self.transformer_encoder(x)
        transformer_embedding = torch.mean(transformer_output, dim = 0)

        complete_embedding = torch.cat([conv2d_embedding1, conv2d_embedding2, transformer_embedding], dim = 1)
        fc1 = self.fc1_layer(complete_embedding)
        ac1 = self.act1(fc1)
#         fc2 = self.fc2_layer(ac1)
#         ac2 = self.act1(fc2)
        output_logits = self.fc2_layer(ac1)
        output_softmax = self.softmax_out(output_logits)
        return output_logits, output_softmaxz

In [ ]:
# def distribute_model(model):
#     if torch.cuda.device_count() < 2:
#         raise ValueError("At least 2 GPUs are required for this operation.")
#     model = nn.DataParallel(model)
#     return model

In [ ]:
# Training step
def make_train_step(model, criterion, optimizer):
    def train_step(X, Y):
        output_logits, output_softmax = model(X)
        predictions = torch.argmax(output_softmax, dim=1)
        Y = torch.argmax(Y, dim=1)
        accuracy = torch.sum(Y==predictions)/float(len(Y))
        loss = criterion(output_logits, Y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item(), accuracy*100
    return train_step

#validation step
def make_validate_fnc(model, criterion):
    def validate(X,Y):
        with torch.no_grad():
            model.eval()
            output_logits, output_softmax = model(X)
            predictions = torch.argmax(output_softmax, dim=1)
            Y = torch.argmax(Y, dim=1)
            accuracy = torch.sum(Y==predictions)/float(len(Y))
            loss = criterion(output_logits, Y)
        return loss.item(), accuracy*100
    return validate

In [ ]:
def load_model(model_path):
    model = define_model(3)
    model.eval()
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
#     model.load_state_dict(torch.load(model_path))
    return model

In [ ]:
def make_save_checkpoint():
    def save_checkpoint(optimizer, model, epoch, filename):
        checkpoint_dict = {
            'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'epoch': epoch
        }
        torch.save(checkpoint_dict, filename)
    return save_checkpoint

def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch, model

In [ ]:
emotions_dict ={
    '0':'positive',
    '1':'neutral',
    '2':'negative'}

# Define optimizer
def choose_optimizer(kind, learning_rate, w_decay, model):
  if kind == 'SGD':
    return torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=w_decay, momentum=0.3)
  if kind == 'ADAM':
    return torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=w_decay, eps = 1e-08)
# Define loss/ criterion
def criterion(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions, target=targets)

# Define the model and distribute it across the two GPUs
model = define_model(len(emotions_dict))
# model = distribute_model(model)
print('Number of trainable params: ', sum(p.numel() for p in model.parameters()))

lr = 0.01
optimizer = choose_optimizer('SGD',lr,1e-3, model)
save_checkpoint = make_save_checkpoint()
train_step = make_train_step(model, criterion, optimizer)
validate = make_validate_fnc(model, criterion)

# Define your device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device2 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model.to(device)
print(f'GPU model: {torch.cuda.get_device_name(0)}')

# Train your model on multiple GPUs
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model, device_ids=[0, 1])

train_losses = []
valid_losses = []

In [ ]:
def train(optimizer, model, num_epochs, X_train, Y_train, X_valid, Y_valid, list_acc, list_loss):
    train_size = X_train.shape[0]
    minibatch = 16
#     acc_max = 0
    for epoch in range(num_epochs):
        #set to train phase
        model.train()

        train_indices = np.random.permutation(train_size)
        X_train = X_train[train_indices,:,:,:]
        Y_train = Y_train[train_indices,:]

        epoch_acc = 0
        epoch_loss = 0
        num_iterations = int(train_size/minibatch)

        for i in range(num_iterations+1):
            batch_start = i*minibatch
            batch_end = min(batch_start + minibatch, train_size)
            actual_batch_size = batch_end - batch_start

            X = X_train[batch_start:batch_end,:,:,:]
            Y = Y_train[batch_start:batch_end,:]

            X_tensor = torch.tensor(X, device = device).float()
            Y_tensor = torch.tensor(Y, dtype = torch.long, device = device)

            loss, acc = train_step(X_tensor, Y_tensor)

            epoch_acc += acc * actual_batch_size / train_size
            epoch_loss += loss * actual_batch_size / train_size

        X_valid_tensor = torch.tensor(X_valid, device = device).float()
        Y_valid_tensor = torch.tensor(Y_valid, dtype = torch.long, device = device)
        valid_loss, valid_acc, = validate(X_valid_tensor,Y_valid_tensor)
        list_acc[0].append(epoch_acc.cpu().numpy())
        list_acc[1].append(valid_acc.cpu().numpy())
        list_loss[0].append(epoch_loss)
        list_loss[1].append(valid_loss)
#         if acc_max > int(epoch_acc):
#           continue
#         acc_max = epoch_acc
#         checkpoint_filename = 'cnn_transformerFINAL-{:03d}.pkl'.format(epoch)
#         save_checkpoint(optimizer, model, epoch, checkpoint_filename)
        torch.save(model.state_dict(),'model'+str(epoch)+'.pt')

        print(f'\nEpoch {epoch} --- loss:{epoch_loss:.3f}, Epoch accuracy:{epoch_acc:.2f}%, Validation loss:{valid_loss:.3f}, Validation accuracy:{valid_acc:.3f}')

In [ ]:
# num_epochs = 100
# list_acc = [[],[]]
# list_loss = [[],[]]
# train(optimizer, model, num_epochs, X_train, Y_train, X_valid[:1000,:,:,:], Y_valid[:1000,:], list_acc, list_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(num_epochs),list_acc[0])
plt.plot(range(num_epochs),list_acc[1])
plt.xlabel('epoch')
plt.ylabel('accuracy')
title = 'SGD lr:' + str(lr) + 'weight_decay:1e-3'
plt.title(title)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(num_epochs),list_loss[0])
plt.plot(range(num_epochs),list_loss[1])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title(title)

## Test

In [ ]:
def load_model(model_path):
    model = define_model(3)
    model.eval()
#     model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.load_state_dict(torch.load(model_path))
    return model

In [ ]:
def metric(y, y_hat):
  # Compute F1 score for each label
    labels = [0,1,2]
    emotions = {0: 'positive',
               1: 'neutral',
               2: 'negative'}
    f1_scores_per_label = [f1_score(y, y_hat, labels=[label], average='weighted') for label in labels]

    # Compute average F1 score across all labels
    average_f1_score = np.mean(f1_scores_per_label)

    # Display F1 scores for each label and the average F1 score
    for label in emotions.keys():
        print("F1 scores for each label:",emotions[label],':', f1_scores_per_label[label])
    print("Average F1 score:", average_f1_score)
    
    accuracy = torch.sum(y==y_hat)/float(len(y))
#     print('F1_score of every label: ', labels)
#     print('Average F1: ',round(statistics.mean(labels.values()),2))
    print('Accuracy: ', accuracy)

In [ ]:
def test(X_test,Y_test):
    model = load_model('/kaggle/working/model98.pt')
    model.eval()

    X_test_tensor = torch.tensor(X_test).float()
    Y_test_tensor = torch.tensor(Y_test, dtype = torch.long)
    
    minibatch = 128
    num_iterations = int(X_test.shape[0]/minibatch)
    list_Y = torch.argmax(Y_test_tensor, dim=1)
    list_Yhat = torch.tensor(())
    for i in range(num_iterations+1):
        batch_start = i*minibatch
        batch_end = min(batch_start + minibatch, X_test.shape[0])
        actual_batch_size = batch_end - batch_start
        
        X = X_test[batch_start:batch_end,:,:,:]
        X_tensor = torch.tensor(X).float()

        output_logits, output_softmax = model(X_tensor)
        predictions = torch.argmax(output_softmax, dim=1)

        list_Yhat = torch.cat((list_Yhat, predictions), dim = 0)
    
    metric(list_Y, list_Yhat)
    return predictions

In [ ]:
Y_hat = test(X_test, Y_test)

In [ ]:
# from sklearn.metrics import roc_auc_score
# import numpy as np
# from sklearn.preprocessing import label_binarize

# # Binarize the true labels for each class
# Y_binarized = label_binarize(Y_test, classes=[0, 1, 2])

# # Compute AUC-ROC for each class
# auc_scores_per_class = [roc_auc_score(Y_binarized[:, i], Y_hat == i) for i in range(Y_binarized.shape[1])]
# auc_scores_per_class